In [1]:
import numpy as np
import numpy.linalg as lin

np.set_printoptions(precision=3, linewidth=150, suppress=True)

## Conditionnement d'une matrice

Soit la matrice A suivante :

In [2]:
A = np.array([[10, 7, 8, 7], [7, 5, 6, 5], [8, 6, 10, 9], [7, 5, 9, 10]])
A

array([[10,  7,  8,  7],
       [ 7,  5,  6,  5],
       [ 8,  6, 10,  9],
       [ 7,  5,  9, 10]])

Une matrice symétrique qui n'a rien de méchant a priori. Son déterminant est 1.

In [3]:
lin.det(A)

0.9999999999999889

Construisons **b** de telle sorte que la solution du système matriciel A **x** = **b** soit [1,1,1,1] :

In [4]:
b = A.sum(axis=1)
print(b)
x = lin.solve(A, b)
x

[32 23 33 31]


array([1., 1., 1., 1.])

Perturbons légèrement **b**. Dans le cas d'une expérience cela s'appelle une erreur de mesure. En informatique
cela peut être le résultat d'erreurs d'arrondi.

In [5]:
bp = [32.1, 22.9, 33.1, 30.9]
eb = lin.norm(b - bp) / lin.norm(b) # une erreur se mesure par rapport à la valeur de la donnée
eb

0.0033319453118976702

On a une erreur sur **b** de l'ordre de 0,3 %. On la note $ ||{\bf \delta b}|| \, / \,||{\bf b}||$.

On pourrait espérer une erreur sur le résultat du même ordre de
grandeur.
Regardons la solution **x** de notre système matriciel perturbé :

In [6]:
xp = lin.solve(A, bp)
xp

array([  9.2, -12.6,   4.5,  -1.1])

Cette solution n'a rien à voir avec [1, 1, 1, 1] !

In [7]:
ex = lin.norm(x - xp) / lin.norm(x)
ex

8.198475468036927

L'erreur est de l'ordre de 8 pour 1.

In [8]:
ex / eb

2460.567236431495

L'erreur est 2460 fois plus grande que l'erreur sur **b** !?

### Pourquoi ?

On a 

$$
\begin{align}
& A ({\bf x} + {\bf \delta x}) = {\bf b} + {\bf \delta b} \quad \textrm{et donc} \\
& A \, {\bf \delta x} = {\bf \delta b} \; \textrm{ puisque } A {\bf x} = {\bf b} \quad \textrm{et finalement}\\
& {\bf \delta x} = A^{-1} \, {\bf \delta b}
\end{align}
$$

Comme A et son inverse sont des applications linéaires on a

$$
\begin{align}
& ||{\bf b}|| \le ||A|| \, ||{\bf x}||
\quad \textrm{et} \quad ||{\bf \delta x}|| \le ||A^{-1}|| \, ||{\bf \delta b}||
\end{align}
$$

donc 

$$
\begin{align}
\frac{||{\bf \delta x}||}{||{\bf x}||}  \le ||A^{-1}|| \, \frac{||{\bf \delta b}||}{||{\bf x}||}
\le ||A^{-1}|| \, ||A|| \, \frac{||{\bf \delta b}||}{||{\bf b}||}
\end{align}
$$

In [9]:
lin.norm(lin.inv(A)) * lin.norm(A)

3009.578708058688

Le problème est là.

On appelle cela le conditionnement de A :
    
$cond(A) = ||A^{-1}|| \, ||A||$

**Une matrice mal conditionnée va générer des erreurs de calcul lors de la résolution du système matriciel.**

In [12]:
lin.cond(A) # scipy n'a pas le conditionnement mais numpy l'a. 

2984.092701675755

Une valeur légèrement différente, mais toujours très grande.

In [13]:
print(A)
Ainv=lin.inv(A)
print(Ainv)
print(lin.norm(A), lin.norm(Ainv))

[[10  7  8  7]
 [ 7  5  6  5]
 [ 8  6 10  9]
 [ 7  5  9 10]]
[[ 25. -41.  10.  -6.]
 [-41.  68. -17.  10.]
 [ 10. -17.   5.  -3.]
 [ -6.  10.  -3.   2.]]
30.54504869860253 98.52918349402991


Il y a des grands nombres dans l'inverse de $A$ ...

### Propriétés

* $cond(A) \ge 1$ car $Id = A\, A^{-1}$ et donc $1 \le ||A||\, ||A^{-1}||$ (pour les normes subordonnées car $||Id||_F = \sqrt{n}$)
* $cond(A) = cond(A^{-1})$ par définition du conditionnement
* $\displaystyle cond_2(A) = \frac{\max_i |\lambda_i|}{\min_i |\lambda_i|}$,
où le 2 indique qu'on utilise la norme 2 et $\lambda_i$ sont les valeurs propres de $A$

In [14]:
print(lin.cond(A))
vp = lin.eigvals(A)
vp.max() / vp.min()

2984.092701675755


2984.092701676269

* si $A$ est unitaire ou orthogonale alors $cond_2(A) = 1$ (une rotation ou symétrie ne va pas agrandir l'erreur)
* le conditionnement n'est pas modifié par transformation unitaire 

## Les pires matrices du monde

In [20]:
import scipy as sp # numpy.linalg ne l'a pas
H4 = sp.linalg.hilbert(4)
print(H4)

[[1.    0.5   0.333 0.25 ]
 [0.5   0.333 0.25  0.2  ]
 [0.333 0.25  0.2   0.167]
 [0.25  0.2   0.167 0.143]]


En général, la matrice de Hilbert de rang $n$ est définie comme
$$H_n =
\begin{bmatrix}
1 & \frac 1 2 & \frac 1 3 & \cdots & \frac 1 n
\\
\frac 1 2 & \frac 1 3 & \frac 1 4 & \cdots & \frac 1 {n+1}
\\
\vdots & \vdots & \vdots & \ddots & \vdots
\\
\frac 1 n & \frac 1 {n+1} & \frac 1 {n+2} & \cdots & \frac 1 {2n-1}
\end{bmatrix}
$$
c. à d. $(H_n)_{i j}=\frac 1 {i+j-1}$

In [21]:
lin.cond(H4)

15513.738738928138

Oups !

In [22]:
lin.inv(H4)

array([[   16.,  -120.,   240.,  -140.],
       [ -120.,  1200., -2700.,  1680.],
       [  240., -2700.,  6480., -4200.],
       [ -140.,  1680., -4200.,  2800.]])

Des grands nombres dans l'inverse !

In [32]:
for n in range(1,12):
    Hn = sp.linalg.hilbert(n)
    Hninv = lin.inv(Hn)
    print(n, lin.cond(Hn))

1 1.0
2 19.28147006790397
3 524.0567775860644
4 15513.738738928138
5 476607.25024172297
6 14951058.641005432
7 475367356.42350554
8 15257575270.772364
9 493154205797.1792
10 16025326851680.766
11 522508642595288.75


Cela grandit rapidemment.  Et, pire encore :

In [42]:
H20 = sp.linalg.hilbert(20)
H20inv = lin.inv(H20)
H20@H20inv

array([[ 1.   ,  0.   ,  0.   ,  0.001,  0.003,  0.006,  0.078,  0.016,  0.062,  0.125,  0.25 ,  0.   ,  0.25 ,  0.203,  0.   , -0.625,  0.   ,
        -0.75 , -0.25 , -0.047],
       [ 0.   ,  1.   , -0.   ,  0.003, -0.   , -0.047, -0.109,  0.156, -0.219,  0.5  ,  0.562, -0.75 ,  0.688,  0.336,  0.375, -2.25 ,  0.375,
        -0.125,  0.   , -0.094],
       [-0.   ,  0.   ,  1.   ,  0.002,  0.021, -0.027, -0.102,  0.172,  0.188,  0.281,  1.   ,  1.5  ,  0.375,  0.305, -0.062,  0.25 ,  1.625,
         3.75 ,  1.812, -0.344],
       [-0.   , -0.   , -0.   ,  0.998,  0.006, -0.039, -0.219, -0.359, -0.109, -0.125, -0.062,  0.125, -1.562,  1.422, -0.312, -1.062,  1.25 ,
        -0.25 ,  1.688,  0.203],
       [-0.   ,  0.   , -0.   ,  0.002,  0.99 , -0.025, -0.281,  0.125,  0.5  ,  0.188,  0.25 ,  0.125, -0.938,  0.125, -0.438, -0.25 ,  0.375,
        -2.   ,  1.25 ,  0.141],
       [-0.   ,  0.   , -0.   ,  0.003, -0.017,  1.012, -0.383, -0.234,  0.328, -0.062,  0.25 ,  1.25 , -1.125, -0.

In [46]:
lin.norm(H20@H20inv) # la norme d'une matrice d'identité est 1, normalement ...

14.01294476768698

N'importe quoi !

## Préconditionnement

Si le conditionnement n'est pas modifié par une transformation unitaire, il l'est par d'autres transformations.
Ainsi pour toute matrice mal-conditionnée $A$ on peut trouver une autre $B$,
appelée matrice de préconditionnement, t.q. $cond(B A)\ll cond(A)$.

Aussi, au lieu de résoudre $A {\bf x} = {\bf b}$ on résoud $B A {\bf x} = B\, {\bf b}$

Toute la difficulté consiste à trouver une matrice de préconditionnement $B$ qui soit efficace et simple à calculer.